In [2]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import airy, airye, riccati_jn, riccati_yn, iv, kv
import os
from airy_props import *

# dirs = [n for n in os.listdir() if (os.path.isdir(n) and ((n[-2:]=='_p') or (n[-2:]=='_s')))]
# dirs = ['H_test_3_im']
# dirs = ['H_0_plus_s', 'H_0_minus_s', 'H_2_s', 'H_3_p']
dir = 'H_0_plus_s'
r, hls, ddrls = load_matrices(dir)
print(dir)
emin = hls[-1,0,0]
n = len(hls[0])
shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
vd = (hls-shift)/4.637
hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
emax = np.abs(np.array(hfunc(r[0])[-1,-1]))
w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])
es = np.array([1])
smats = {}
for i in range(n):
    emax = np.abs(np.array(hfunc(r[0])[i,i]))
    emin = np.abs(np.array(hfunc(r[-1])[i,i]))
    es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
es = np.unique(np.round(np.sort(es)))
sigmas = np.zeros((len(es), n, n))
ecm = es*4.637+np.real(hls[-1,0,0])
for ei in range(len(es)):
    e = es[ei]
    lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
    ks = np.sqrt(np.abs(np.diag(e*np.eye(n)-hfunc(r[-1]))))
    yf = np.zeros((lmax, n, n), dtype=complex)
    nt = np.sum(np.diag(vd[-1]) < e)
    for l in range(lmax):
        yf[l] = airy_imb_prop(r, w(r[:-1]+np.diff(r)/2, l, e), wp(r[:-1]+np.diff(r)/2, l), n)
    ytot = np.array([(yf[l]+np.matrix(yf[l]).H)/2 for l in range(lmax)])
    jmat = np.zeros((lmax, n, n))
    nmat = np.zeros((lmax, n, n))
    jpmat = np.zeros((lmax, n, n))
    npmat = np.zeros((lmax, n, n))
    for j in range(n):
        if j <= nt:
            prej, prejp = riccati_jn(lmax-1, np.real(ks[j])*r[-1])
            pren, prenp = riccati_yn(lmax-1, np.real(ks[j])*r[-1])
            jmat[:,j,j] = prej/np.sqrt(np.real(ks[j]))
            nmat[:,j,j] = pren/np.sqrt(np.real(ks[j]))
            jpmat[:,j,j] = np.sqrt(np.real(ks[j])) * prejp
            npmat[:,j,j] = np.sqrt(np.real(ks[j])) * prenp
        else:
            jmat[:,j,j], prejp = riccati_in(lmax-1, np.real(ks[j])*r[-1])
            nmat[:,j,j], prenp = riccati_kn(lmax-1, np.real(ks[j])*r[-1])
            jpmat[:,j,j] = ks[j] * prejp
            npmat[:,j,j] = ks[j] * prenp
    rmat = np.array([-np.matrix(npmat[l] - ytot[l] @ nmat[l]).I @ np.matrix(jpmat[l] - ytot[l] @ jmat[l]) for l in range(lmax)])
    smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
    print(smat[0])
    smats[f'E = {ecm[ei]}'] = smat

H_0_plus_s
[[-0.8114896+0.58436685j]]
[[-0.42429044-0.90552616j]]
[[0.51855941+0.8550416j]]
[[-0.03357322-0.99943626j]]
[[-0.75264762+0.65842354j]]
[[0.90609184+0.42308106j]]
[[0.22420351-0.97454235j]]
[[-0.98521168-0.17134162j]]
[[-0.24739408+0.96891495j]]
[[0.90691804+0.42130709j]]
[[0.65186209-0.75833753j]]
[[-0.4872434-0.87326621j]]
[[-0.99558097+0.09390707j]]
[[-0.36746419+0.93003767j]]
[[0.62345462+0.78185954j]]
[[0.99434538-0.10619443j]]
[[0.48046717-0.87701271j]]
[[-0.41092662-0.91166842j]]
[[-0.9667014-0.25590702j]]
[[-0.82696964+0.56224657j]]
[[-0.14680723+0.98916512j]]
[[0.60674096+0.79489962j]]
[[0.98963471+0.14360761j]]
[[0.82053344-0.57159852j]]
[[0.22836253-0.97357617j]]
[[-0.46338915-0.8861549j]]
[[-0.92274469-0.38541178j]]
[[-0.96111006+0.27616564j]]
[[-0.59120374+0.80652225j]]
[[0.00947467+0.99995511j]]
[[0.59374225+0.8046553j]]
[[0.94803019+0.31818038j]]
[[0.96282884-0.27011224j]]
[[0.65413026-0.75638192j]]
[[0.14055631-0.99007269j]]
[[-0.40734449-0.91327458j]]
[[-0.